# Todo
- [x] load clip
- [x] load images
- [x] batch run inference on images
- [x] figure out how to load images from zip
- [x] save embeddings
- [x] benchmark, gpu: 2h
- [x] upload to gcp bucket
- [x] embed text as well
- [ ] predict a baseline score for kaggle, on image+description cos similarity only

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May  9 19:58:28 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import auth
auth.authenticate_user()

Don't forget to upload your **kaggle.json** for authentication

# Setup

1. get data

In [ ]:
!KAGGLE_CONFIG_DIR=/content kaggle competitions download -c h-and-m-personalized-fashion-recommendations

100% 28.7G/28.7G [05:28<00:00, 130MB/s] 
100% 28.7G/28.7G [05:29<00:00, 93.6MB/s]


2. mount zip

In [ ]:
!apt-get install -y fuse-zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following additional packages will be installed:
  libzip4
The following NEW packages will be installed:
  fuse-zip libzip4
0 upgraded, 2 newly installed, 0 to remove and 42 not upgraded.
Need to get 65.6 kB of archives.
After this operation, 178 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libzip4 amd64 1.1.2-1.1 [37.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fuse-zip amd64 0.4.4-1 [27.9 kB]
Fetched 65.6 kB in 0s (751 kB/s)
Selecting previously unselected package libzip4:amd64.
(Reading database ... 155203 files and directories currently installed.)
Preparing to unpack .../libzip4_1.1.2-1.1_amd64.deb ...
Unpacking libzip4:amd64 (1.1.2-1.1) ...
S

In [ ]:
!mkdir /content/archive
!fuse-zip /content/h-and-m-personalized-fashion-recommendations.zip /content/archive

In [ ]:
# to unmount
# !fusermount -u /content/archive

3. get clip model

In [ ]:
!pip install torch

In [ ]:
!pip install torch-scatter torch-sparse -f https://data.pyg.org/whl/torch-1.11.0+cu113.html

Looking in links: https://data.pyg.org/whl/torch-1.11.0+cu113.html
     |████████████████████████████████| 7.9 MB 14.4 MB/s 
     |████████████████████████████████| 3.5 MB 60.3 MB/s 


In [ ]:
# !pip uninstall -q -y torch-sparse
# !pip uninstall -q -y torch-scatter

In [ ]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git
!pip install https://github.com/pyg-team/pytorch_geometric/archive/master.zip

     |████████████████████████████████| 53 kB 1.6 MB/s 
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ei8l_pcc
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-ei8l_pcc
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=ce2f349f249d5f7bccf4c4b733d23f19659461d28083cc7ca172b515f7bea754
  Stored in directory: /tmp/pip-ephem-wheel-cache-x8j1543z/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip
     / 2.4 MB 3.9 MB/s
  Created wheel for torch-geometric: filename=torch_geometric-2.0.5-py3-none-any.whl size=636898 sha256=5dce419fa7f1c5eccdcae213abc57b9d5611cbbd06302b3010d35b9b95664902
  Stored in directory: /tmp/pip-ephem-wheel-cache-xs1jn4tf/wheels/70/53/71/38e50390ffab43b7bf5e55f1cdec398bbb09b9b3d2facb4478
Successfully built torch-geometric


# Get embeddings

In [ ]:
import torch
import clip

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# TODO to use tpu
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [ ]:
# example useage of clip ala. https://github.com/openai/CLIP
# model, preprocess = clip.load("ViT-B/32", device=device)
# image = preprocess(Image.open("/content/images/010/0108775015.jpg")).unsqueeze(0).to(device)
# text = clip.tokenize(["a dress", "a dog", "a cat"]).to(device)

# with torch.no_grad():
#     image_features = model.encode_image(image)
#     text_features = model.encode_text(text)

#     logits_per_image, logits_per_text = model(image, text)
#     probs = logits_per_image.softmax(dim=-1).cpu().numpy()

# print("Label probs:", probs)

In [ ]:
import pandas as pd
import zipfile
from io import BytesIO
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from os.path import exists


class FashionImagesDataset(Dataset):
    def __init__(self, transform=lambda id: id):
        self.articles = pd.read_csv('/content/archive/articles.csv')
        self.articles['img_path'] = self.articles['article_id'].map(lambda id: "/content/archive/images/0" + str(id)[0:2] + "/0" + str(id) + ".jpg")
        self.valid_idx = self.articles[self.articles.apply(lambda article: exists(article['img_path']), axis=1)]
        print('valid and has image:', len(self.valid_idx), 'from:', len(self.articles))
        self.transform = transform

    def __len__(self):
        return len(self.valid_idx)

    def __getitem__(self, idx):
        img_path = self.valid_idx.iloc[idx]['img_path']
        image = Image.open(img_path)
        label = self.valid_idx.iloc[idx]['article_id']
        image = self.transform(image)
        return image, label

In [ ]:
model_name = 'ViT-B/32'
# also ViT-L/14, etc.
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14']

In [ ]:
model, preprocess = clip.load(model_name, device=device)

100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 195MiB/s]


In [ ]:
batch_size = 64

In [ ]:
dataset = FashionImagesDataset(transform=preprocess)

valid and has image: 105100 from: 105542


In [ ]:
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [ ]:
images, labels = next(iter(data_loader))

In [ ]:
images.size(), images.chunk(batch_size)[1].squeeze().size()

(torch.Size([64, 3, 224, 224]), torch.Size([3, 224, 224]))

# Get image embeddings

In [ ]:
from tqdm import tqdm

image_features = {}
with torch.no_grad():
    for images, labels in tqdm(data_loader):
      features = model.encode_image(images.to(device))
      for label, feature in zip(labels, features):
        image_features[label.item()] = feature.to('cpu')

100%|██████████| 1643/1643 [1:20:46<00:00,  2.95s/it]


In [ ]:
image_features[111565003].size()

torch.Size([512])

# Save image embeddings

In [ ]:
file_name = '/content/fashion-recommendation-image-embeddings-clip-' + model_name.replace('/', '-') + '.pt'

In [ ]:
torch.save(image_features, file_name)

In [ ]:
len(image_features.keys())

105100

In [ ]:
!ls -lah $file_name

-rw-r--r-- 1 root root 130M Apr  5 14:57 /content/fashion-recommendation-image-embeddings-clip-ViT-B-32.pt


In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gsutil cp $file_name gs://heii-public/

Copying file:///content/fashion-recommendation-image-embeddings-clip-ViT-B-32.pt [Content-Type=application/octet-stream]...
|
Operation completed over 1 objects/130.0 MiB.                                    


# Reload image embeddings (e.g.: if you restart colab, etc...)



In [ ]:
file_name = '/content/fashion-recommendation-image-embeddings-clip-' + model_name.replace('/', '-') + '.pt'

In [ ]:
remote_file = "https://storage.googleapis.com/heii-public/" + file_name.replace('/content/', '')
remote_file

In [ ]:
!wget $remote_file

In [ ]:
image_features = torch.load(file_name)

# Get text embeddings

In [ ]:
articles = pd.read_csv('/content/archive/articles.csv')
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [ ]:
keys = ['derived_name', 'derived_look', 'derived_category', 'prod_name', 'product_type_name', 'product_group_name', 'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name', 'department_name', 'index_name', 'index_group_name', 'section_name', 'garment_group_name', 'detail_desc']

In [ ]:
articles = pd.read_csv('/content/archive/articles.csv')
articles['derived_name'] = articles.apply(lambda row: ' '.join(['This is a', row['prod_name'], row['product_type_name']]), axis=1)
articles['derived_look'] = articles.apply(lambda row: ' '.join(['It has a', row['graphical_appearance_name'], row['perceived_colour_value_name'], row['colour_group_name'], 'color']), axis=1)
articles['derived_category'] = articles.apply(lambda row: ' / '.join([row['index_group_name'], row['index_name'], row['section_name'], row['department_name'], row['garment_group_name'], row['product_group_name'], row['product_type_name'], row['graphical_appearance_name'], row['colour_group_name']]), axis=1)
print('Example derived values:')
for i in range(3):
    print(articles.iloc[i]['derived_name'])
    print(articles.iloc[i]['derived_look'])
    print(articles.iloc[i]['derived_category'])

Example derived values:
This is a Strap top Vest top
It has a Solid Dark Black color
Ladieswear / Ladieswear / Womens Everyday Basics / Jersey Basic / Jersey Basic / Garment Upper body / Vest top / Solid / Black
This is a Strap top Vest top
It has a Solid Light White color
Ladieswear / Ladieswear / Womens Everyday Basics / Jersey Basic / Jersey Basic / Garment Upper body / Vest top / Solid / White
This is a Strap top (1) Vest top
It has a Stripe Dusty Light Off White color
Ladieswear / Ladieswear / Womens Everyday Basics / Jersey Basic / Jersey Basic / Garment Upper body / Vest top / Stripe / Off White


In [ ]:
class FashionTextDataset(Dataset):
    def __init__(self, key, articles):
        self.key = key
        self.articles = articles

    def __len__(self):
        return len(self.articles)

    def __getitem__(self, idx):
        label = self.articles.iloc[idx]['article_id']
        # tokenize already pads text
        tokens = clip.tokenize(str.strip(str(self.articles.iloc[idx][self.key])), 77, True).squeeze().to(device)
        return tokens, label

In [ ]:
text_dataset = FashionTextDataset(key=keys[0], articles=articles)

In [ ]:
text_data_loader = DataLoader(text_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
tokens, labels = next(iter(text_data_loader))

In [ ]:
batch_size, len(keys), tokens.size()

(64, 15, torch.Size([64, 77]))

In [ ]:
tokenizer = clip.simple_tokenizer.SimpleTokenizer()
tokenizer.decode(tokens[1].tolist())

'<|startoftext|>this is a strap top vest top <|endoftext|>!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!'

In [ ]:
from tqdm import tqdm

text_features = {}
with torch.no_grad():
  for i, key in enumerate(keys):
      print('getting embeddings for ', key, 'text fields left:', len(keys)-i)
      text_dataset_ = FashionTextDataset(key=key, articles=articles)
      text_data_loader_ = DataLoader(text_dataset_, batch_size=64, shuffle=False)
      for tokens, labels in tqdm(text_data_loader_):
          features = model.encode_text(tokens)
          for label, feature in zip(labels, features):
              text_features.setdefault(label.item(), {})[key] = feature.to('cpu')

getting embeddings for  derived_name text fields left: 15


100%|██████████| 1650/1650 [03:22<00:00,  8.16it/s]


getting embeddings for  derived_look text fields left: 14


100%|██████████| 1650/1650 [03:22<00:00,  8.15it/s]


getting embeddings for  derived_category text fields left: 13


...

100%|██████████| 1650/1650 [03:17<00:00,  8.33it/s]


getting embeddings for  garment_group_name text fields left: 2


100%|██████████| 1650/1650 [03:17<00:00,  8.37it/s]


getting embeddings for  detail_desc text fields left: 1


100%|██████████| 1650/1650 [03:31<00:00,  7.82it/s]


In [ ]:
text_features[116379047]['derived_name'].size()

torch.Size([512])

# Save text embeddings

should match len(articles):

In [ ]:
len(articles), len(text_features.keys()), len(articles) == len(text_features.keys()), all([512 == len(text_features[108775015].get(key, {})) for key in keys])

(105542, 105542, True, True)

In [ ]:
text_file_name = '/content/fashion-recommendation-text-embeddings-clip-' + model_name.replace('/', '-') + '.pt'

In [ ]:
torch.save(text_features, text_file_name)

In [ ]:
!gsutil cp $text_file_name gs://heii-public/

Copying file:///content/fashion-recommendation-text-embeddings-clip-ViT-B-32.pt [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

-
Operation completed over 1 objects/1.9 GiB.                                      


In [ ]:
remote_text_file = "https://storage.googleapis.com/heii-public/" + text_file_name.replace('/content/', '')
remote_text_file

'https://storage.googleapis.com/heii-public/fashion-recommendation-text-embeddings-clip-ViT-B-32.pt'

# Reload text embeddings

In [ ]:
text_file_name = '/content/fashion-recommendation-text-embeddings-clip-' + model_name.replace('/', '-') + '.pt'

In [ ]:
remote_text_file = "https://storage.googleapis.com/heii-public/" + text_file_name.replace('/content/', '')

In [ ]:
!wget $remote_text_file

--2022-04-05 11:33:10--  https://storage.googleapis.com/heii-public/fashion-recommendation-text-embeddings-clip-ViT-B-32.pt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.5.128, 74.125.133.128, 108.177.15.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.5.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 480994078 (459M) [application/octet-stream]
Saving to: ‘fashion-recommendation-text-embeddings-clip-ViT-B-32.pt’

fashion-recommendat 100%[===================>] 458.71M  95.1MB/s    in 5.2s    

2022-04-05 11:33:16 (88.5 MB/s) - ‘fashion-recommendation-text-embeddings-clip-ViT-B-32.pt’ saved [480994078/480994078]



In [ ]:
text_features = torch.load(text_file_name)

# Predict customer - product interaction:
prerequisites: run setup

In [ ]:
import torch
import pandas as pd
from sklearn import preprocessing
import torch_geometric.transforms as T
from torch_geometric.data import HeteroData, InMemoryDataset, download_url
from torch_geometric.utils import negative_sampling


class HMDataset(InMemoryDataset):
    image_embeddings_url = "https://storage.googleapis.com/heii-public/fashion-recommendation-image-embeddings-clip-ViT-B-32.pt"
    text_embeddings_url = "https://storage.googleapis.com/heii-public/fashion-recommendation-text-embeddings-clip-ViT-B-32.pt"
    raw_dir = "/content"
    processed_dir = "/content"

    def __init__(self, root, transform=None, pre_transform=None):
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self):
        return [
            "fashion-recommendation-image-embeddings-clip-ViT-B-32.pt",
            "fashion-recommendation-text-embeddings-clip-ViT-B-32.pt",
            "archive/articles.csv",
            "archive/customers.csv",
            "archive/transactions_train.csv",
        ]

    @property
    def processed_file_names(self):
        return f"hm_graph.pt"

    def download(self):
        download_url(self.image_embeddings_url, self.raw_dir)
        download_url(self.text_embeddings_url, self.raw_dir)

    def process(self):
        self.articles = pd.read_csv(self.raw_paths[2], index_col="article_id")
        self.customers = pd.read_csv(self.raw_paths[3], index_col="customer_id").fillna(
            0.0
        )
        self.transactions = pd.read_csv(self.raw_paths[4])

        data = HeteroData()

        # create node edges
        t = self.transactions.to_dict()
        customers_id_ix = {v: k for k, v in enumerate(self.customers.index.unique())}
        # customers_ix_id = {k: v for k, v in enumerate(self.customers.index.unique())}
        articles_id_ix = {v: k for k, v in enumerate(self.articles.index.unique())}
        # articles_ix_id = {k: v for k, v in enumerate(self.articles.index.unique())}
        src = [customers_id_ix[t["customer_id"][i]] for i in t["customer_id"]]
        dst = [articles_id_ix[t["article_id"][i]] for i in t["article_id"]]
        data["customer", "buys", "article"].edge_index = torch.tensor([src, dst]).long()

        # avoid out of memory on colab
        del t
        del customers_id_ix
        del articles_id_ix

        # encode customers
        le = preprocessing.LabelEncoder()
        self.customers["postal_code"] = le.fit_transform(self.customers["postal_code"])
        self.customers.loc[
            self.customers["fashion_news_frequency"] == "None", "fashion_news_frequency"
        ] = 0.0
        self.customers.loc[
            self.customers["fashion_news_frequency"] == "NONE", "fashion_news_frequency"
        ] = 0.0
        customer_features = self.customers[
            [
                "postal_code",
                "age",
                "fashion_news_frequency",
                "FN",
                "Active",
                "club_member_status",
            ]
        ]
        customer_features = pd.get_dummies(
            customer_features,
            columns=["age", "fashion_news_frequency", "club_member_status"],
        )
        customer_features = torch.from_numpy(customer_features.to_numpy())

        # encode articles
        self.article_image_embeddings = torch.load(self.raw_paths[0])
        self.article_text_embeddings = torch.load(self.raw_paths[1])
        self.articles = self.articles.merge(
            self.transactions.groupby("article_id")["price"].mean(),
            on="article_id",
            how="outer",
        ).fillna(0.0)
        # self.articles["price_bin"] = pd.qcut(self.articles["price"], 100, labels=False)
        self.articles["product_type_no"] = self.articles["product_type_no"].astype(str)
        product_type_no_le = preprocessing.LabelEncoder()
        self.articles["product_type_no"] = product_type_no_le.fit_transform(
            self.articles["product_type_no"]
        )
        self.articles["graphical_appearance_no"] = self.articles[
            "graphical_appearance_no"
        ].astype(str)
        graphical_appearance_no_le = preprocessing.LabelEncoder()
        self.articles[
            "graphical_appearance_no"
        ] = graphical_appearance_no_le.fit_transform(
            self.articles["graphical_appearance_no"]
        )
        article_features = self.articles[
            ["product_type_no", "graphical_appearance_no", "price"]
        ]
        # article_features = pd.get_dummies(
        #     article_features,
        #     columns=["price_bin"],
        # )
        article_features = torch.from_numpy(article_features.to_numpy())
        article_features = torch.cat(
            (
                article_features,
                torch.stack(
                    self.articles.apply(
                        lambda article: self.article_image_embeddings.get(
                            int(article.name), torch.zeros(512)
                        ),
                        axis=1,
                    ).tolist()
                ),
            ),
            1,
        )
        for key in ["derived_name", "derived_look", "derived_category"]:
            article_features = torch.cat(
                (
                    article_features,
                    torch.stack(
                        self.articles.apply(
                            lambda article: self.article_text_embeddings[
                                int(article.name)
                            ].get(key, torch.zeros(512)),
                            axis=1,
                        ).tolist()
                    ),
                ),
                1,
            )

        # create nodes
        data["article"].x = article_features.float()
        data["customer"].x = customer_features.float()

        # transform?
        if self.pre_transform is not None:
            data = self.pre_transform(data)

        # PyTorch tensor functionality:
        # data = data.pin_memory()
        # data = data.to('cuda:0', non_blocking=True)
        torch.save(self.collate([data]), self.processed_paths[0])


if __name__ == "__main__":
    dataset = HMDataset("/content")

In [ ]:
import math
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.nn import Linear, ModuleList
from torch_geometric.loader import LinkNeighborLoader
import torch_geometric.transforms as T
# from hm_dataset import HMDataset
from torch_geometric.nn import GCN2Conv, GraphConv, SAGEConv, GATConv, to_hetero


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()

dataset = HMDataset("/content")
data = dataset[0] # .to(device)
data["article"].x = data["article"].x.float()
data["customer"].x = data["customer"].x.float()
data[("customer", "buys", "article")].edge_index = data[
    ("customer", "buys", "article")
].edge_index.long()

# Add a reverse ('article', 'rev_buys', 'customer') relation for message passing:
data = T.Compose([T.ToUndirected(), T.NormalizeFeatures()])(data).to('cpu')

# Perform a link-level split into training, validation, and test edges:
train_data, val_data, test_data = T.RandomLinkSplit(
    num_val=0.03,
    num_test=0.95,
    neg_sampling_ratio=0.3,
    add_negative_train_samples=True,
    edge_types=[("customer", "buys", "article")],
    rev_edge_types=[("article", "rev_buys", "customer")],
    is_undirected=True,
)(data)
# when neg_sampling_ratio > 0 and add_negative_train_samples=True only then you will have negative edges

def create_loader(d, shuffle=True):
    return LinkNeighborLoader(
      d,
      num_neighbors=[-1]*2,
      batch_size=1024,
      edge_label_index=(("customer", "buys", "article"), d[("customer", "buys", "article")].edge_label_index),
      edge_label=d[("customer", "buys", "article")].edge_label,
      directed=False,
      replace=False,
      shuffle=shuffle,
      pin_memory=True,
      num_workers=1
    )
train_loader = create_loader(train_data)
val_loader = create_loader(val_data, False)
test_loader = create_loader(test_data)


class GNNEncoder(torch.nn.Module):
    def __init__(self, num_layers, hidden_channels, out_channels):
        super().__init__()
        self.num_layers = num_layers
        # self.convs = ModuleList([GATConv((-1, -1), hidden_channels, heads=4, add_self_loops=False, dropout=0.6).jittable() for i in range(self.num_layers-1)])
        self.conv = GATConv((-1, -1), hidden_channels, heads=6, add_self_loops=False, dropout=0.6)
        self.conv2 = GATConv(hidden_channels*6, out_channels, heads=1, concat=True, add_self_loops=False, dropout=0.6)
        # self.conv3 = SAGEConv((-1, -1), hidden_channels)
        self.out_conv = SAGEConv((-1, -1), hidden_channels)

    def forward(self, x, edge_index):
        # for layer in self.convs:
        #     x = layer(x, edge_index).relu()
        x = self.conv(x, edge_index).relu()
        x = self.conv2(x, edge_index).relu()
        # x = self.conv3(x, edge_index).relu()
        x = self.out_conv(x, edge_index)
        return x


class EdgeDecoder(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.lin1 = Linear(2 * hidden_channels, hidden_channels)
        # self.lin2 = Linear(hidden_channels, hidden_channels)
        # self.lin3 = Linear(hidden_channels, hidden_channels)
        # self.lin4 = Linear(hidden_channels, hidden_channels)
        # self.lin5 = Linear(hidden_channels, hidden_channels)
        self.lin6 = Linear(hidden_channels, 1)

    def forward(self, z_dict, edge_label_index):
        row, col = edge_label_index
        z = torch.cat([z_dict["customer"][row], z_dict["article"][col]], dim=-1)

        z = self.lin1(z).relu()
        # z = self.lin2(z).relu()
        # z = self.lin3(z).relu()
        # z = self.lin4(z).relu()
        # z = self.lin5(z).relu()
        z = self.lin6(z)
        return z.view(-1)


class Model(torch.nn.Module):
    def __init__(self, num_encoder_layers, hidden_channels):
        super().__init__()
        self.encoder = GNNEncoder(num_encoder_layers, hidden_channels, hidden_channels)
        self.encoder = to_hetero(self.encoder, data.metadata(), aggr="sum")
        self.decoder = EdgeDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, edge_label_index):
        z_dict = self.encoder(x_dict, edge_index_dict)
        return self.decoder(z_dict, edge_label_index)


model = Model(num_encoder_layers=1, hidden_channels=32).to(device)

In [ ]:
train_data

HeteroData(
  article={ x=[105542, 2051] },
  customer={ x=[1371980, 95] },
  (customer, buys, article)={
    edge_index=[2, 635768],
    edge_label=[826498],
    edge_label_index=[2, 826498]
  },
  (article, rev_buys, customer)={ edge_index=[2, 635768] }
)

In [ ]:
# model.load_state_dict(torch.load(f"/content/link_pred_0.pt"))

In [ ]:
# !export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:1024

In [ ]:

# Due to lazy initialization, we need to run one model step so the number
# of parameters can be inferred:
with torch.no_grad():
    batch_ = next(iter(train_loader)).to(device, non_blocking=True)
    model.encoder(batch_.x_dict, batch_.edge_index_dict)
    del batch_

optimizer = torch.optim.Adam(model.parameters(), lr=0.03, weight_decay=5e-4)


def train(train_data):
    model.train()
    optimizer.zero_grad()
    pred = model(
        train_data.x_dict,
        train_data.edge_index_dict,
        train_data["customer", "article"].edge_label_index,
    )
    target = train_data["customer", "article"].edge_label
    # loss = F.mse_loss(pred, target)
    loss = F.binary_cross_entropy_with_logits(pred, target)
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(data):
    model.eval()
    pred = model(
        data.x_dict,
        data.edge_index_dict,
        data["customer", "article"].edge_label_index,
    )
    pred = pred.clamp(min=0, max=1)
    target = data["customer", "article"].edge_label
    # loss = F.mse_loss(pred, target).sqrt()
    loss = F.binary_cross_entropy_with_logits(pred, target)
    return float(loss)


torch.cuda.reset_peak_memory_stats()

num_epochs, train_steps, val_steps, test_steps = 301, 2000, 60, 40
for epoch in range(1, num_epochs):
    train_loss = 0
    prog = tqdm(zip(range(train_steps), train_loader), total=train_steps)
    for i, batch in prog:
        batch = batch.to(device, non_blocking=True)
        loss = train(batch)
        train_loss += loss
        # train_rmse = test(batch)
        # cuda_mem = torch.cuda.memory_stats()
        # cuda_reserved = torch.cuda.max_memory_reserved()
        prog.set_description(f"loss: {loss:.4f}")
    train_loss = train_loss / train_steps

    val_loss = 0
    for i, batch in tqdm(zip(range(val_steps), val_loader), total=val_steps):
        val_loss += test(batch.to(device, non_blocking=True))
    val_loss = val_loss / val_steps

    test_loss = 0
    for i, batch in tqdm(zip(range(test_steps), test_loader), total=test_steps):
        test_loss += test(batch.to(device, non_blocking=True))
    test_loss = test_loss / test_steps

    if epoch % 10 == 0:
        torch.save(model.state_dict(), f"/content/link_pred_{epoch:03d}.pt")

    print(
        f"Epoch: {epoch:03d}, Train: {train_loss:.4f}"
        f" Val: {val_loss:.4f}, Test: {test_loss:.4f}"
    )


100%|██████████| 40/40 [00:27<00:00,  1.45it/s]


Epoch: 001, Train: 0.0517 Val: 0.4593, Test: 0.5042


loss: 0.1042:  40%|████      | 808/2000 [03:43<05:29,  3.62it/s]


KeyboardInterrupt: ignored

In [ ]:
torch.save(model.state_dict(), f"/content/link_pred_num_nei_-1_2_epoch_10.pt")

# Link Pred

In [ ]:
# LinkProp and LinkProp-Multi from the paper:
# Revisiting Neighborhood-based Link Prediction for Collaborative Filtering
# https://arxiv.org/abs/2203.15789

import math
import numpy as np
import torch
from torch_sparse import SparseTensor, matmul
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import ndcg_score, recall_score, precision_score, accuracy_score
from itertools import product

device = "cuda" if torch.cuda.is_available() else "cpu"


def load_data():
    try:
        return torch.load("/content/link_prop.pt")
    except FileNotFoundError:
        pass

    customers = pd.read_csv("/content/archive/customers.csv")
    articles = pd.read_csv("/content/archive/articles.csv")
    transactions = pd.read_csv("/content/archive/transactions_train.csv")
    customers.reset_index()
    articles.reset_index()

    src = list(customers["customer_id"])
    src_map = {v: k for k, v in enumerate(src)}
    dest = list(articles["article_id"])
    dest_map = {v: k for k, v in enumerate(dest)}

    transactions.reset_index()
    transactions["src"] = transactions["customer_id"].map(src_map)
    transactions["dest"] = transactions["article_id"].map(dest_map)

    # shuffle transactions
    edge_index = torch.tensor(
        transactions[["src", "dest"]].sample(frac=1).values, dtype=torch.long
    ).T

    # create user-item interaction matrix
    M = SparseTensor(
        row=edge_index[0],
        col=edge_index[1],
        value=torch.ones(len(transactions), dtype=torch.float),
        sparse_sizes=(len(src), len(dest)),
    ).coalesce("max")

    torch.save((M, src, dest, src_map, dest_map), "/content/link_prop.pt")

    return M.to(device), src, dest, src_map, dest_map


def split_data(start, count, M):
    """Selects a range of users"""
    return M[start : start + count, :].coalesce("max")  # .to_dense()


def sparse_assign(m, i, j, val):
    """Sets a value in a sparse matrix at (i,j)"""
    assert i < m.size(0) and j < m.size(1), "can only assign to existing indices"
    row, col, value = m.coo()
    prev = m[i.item(), j.item()].to_dense().squeeze()
    return SparseTensor(
        row=torch.cat((row.to(device), torch.tensor([i], dtype=torch.long).to(device))),
        col=torch.cat((col.to(device), torch.tensor([j], dtype=torch.long).to(device))),
        value=torch.cat(
            (
                value.to(device),
                torch.tensor([val], dtype=torch.float).to(device) - prev.to(device),
            )
        ),
        sparse_sizes=(m.size(0), m.size(1)),
    ).coalesce("add")


def sparse_re_assign_on_mask(m, mask, val, dim=0):
    """Fills masked values in a sparse matrix with a row or col mask. Only sets values that have been explicitly set before."""
    row, col, value = m.coo()
    prev_row, prev_col, prev_value = m[mask, :].coo() if dim == 0 else m[:, mask].coo()
    # restore previous values' indices, assumes mask is boolean and same length as m
    if dim == 0:
        prev_row = torch.arange(len(mask))[mask][prev_row]
    else:
        prev_col = torch.arange(len(mask))[mask][prev_col]
    return SparseTensor(
        row=torch.cat((row.to(device), prev_row.to(device))),
        col=torch.cat((col.to(device), prev_col.to(device))),
        value=torch.cat((value.to(device), val - prev_value.to(device))),
        sparse_sizes=(m.size(0), m.size(1)),
    ).coalesce("add")


def sparse_cat(m, n):
    """Stack two sparse matrices"""
    assert m.size(1) == n.size(
        1
    ), "cannot stack matrices with different number of columns"
    row, col, value = n.coo()
    row = row + m.size(0)
    m_row, m_col, m_value = m.coo()
    return SparseTensor(
        row=torch.cat((m_row.to(device), row.to(device))),
        col=torch.cat((m_col.to(device), col.to(device))),
        value=torch.cat((m_value.to(device), value.to(device))),
        sparse_sizes=(m.size(0) + n.size(0), m.size(1)),
    ).coalesce("add")


def sparse_batch_op(batch_size, op, m, n):
    """Batch runs any torch or numpy operation on 2 sparse matrices with same dims"""
    result = SparseTensor(
        row=torch.tensor([], dtype=torch.long).to(device),
        col=torch.tensor([], dtype=torch.long).to(device),
        value=torch.tensor([], dtype=torch.float).to(device),
        sparse_sizes=(0, m.size(1)),
    )
    for i in range(0, m.size(0), batch_size):
        end = min(i + batch_size, m.size(0))
        res = SparseTensor.from_dense(
            op(
                m[i:end, :].to_dense().to(device),
                n[i:end, :].to_dense().to(device),
            )
        )
        result = sparse_cat(result, res)
    return result.to(device)


def sparse_nonzero(m):
    """Returns the nonzero indices of a sparse matrix"""
    row, col, value = m.clone().coo()
    return (
        row[value != 0].to(device),
        col[value != 0].to(device),
        value[value != 0].to(device),
    )


def sparse_blocked_matmul(m, n, block_size=20000):
    """Dense multiplies a dense and a sparse matrix when you cannot allocate the second matrix all at once"""
    result = torch.empty((m.size(0), 0), dtype=torch.float).to('cpu')
    for block_start in range(0, n.size(1), block_size):
        block_end = min(block_start + block_size, n.size(1))
        res = m.to(device) @ n[:, block_start:block_end].to_dense().to(device)
        result = torch.cat((result, res.to('cpu')), dim=1).to('cpu')
    return result


def intersect2d(a, b):
    """Returns the intersection of two 2D arrays row by row"""
    return np.array([np.intersect1d(a[i], b[i]) for i in range(len(a))])


def sample_user_items(target, ratio=0.4):
    """Drop some edges for users that have more than 1 item"""
    user_deg = target.sum(dim=1)
    candidate = sparse_re_assign_on_mask(target, user_deg > 1, 2, dim=0)
    row, col, value = sparse_nonzero(candidate)
    rand_mask = (value == 2) & (
        torch.rand(len(value)).to(device) < ((value == 2).sum() / len(value) * ratio)
    )
    value[rand_mask] = 0
    value[value == 2] = 1
    data = (
        SparseTensor(
            row=row,
            col=col,
            value=value,
            sparse_sizes=(target.size(0), target.size(1)),
        )
        .coalesce("add")
        .to(device)
    )

    # take observed out of ground truth
    # TODO does this create users with 0 items?
    row, col, value = data.clone().coo()
    value[value == 1] = 2
    value[value == 0] = 1
    value[value == 2] = 0
    target_new_links = (
        SparseTensor(
            row=row,
            col=col,
            value=value,
            sparse_sizes=(target.size(0), target.size(1)),
        )
        .coalesce("add")
        .to(device)
    )

    return data, target_new_links


def mean_average_precision(y_true, y_pred, k=12):
    """Courtesy of https://www.kaggle.com/code/george86/calculate-map-12-fast-faster-fastest"""
    # compute the Rel@K for all items
    rel_at_k = np.zeros((len(y_true), k), dtype=int)

    # collect the intersection indexes (for the ranking vector) for all pairs
    for idx, (truth, pred) in enumerate(zip(y_true, y_pred)):
        _, _, inter_idxs = np.intersect1d(
            truth, pred[:k], assume_unique=True, return_indices=True
        )
        rel_at_k[idx, inter_idxs] = 1

    # Calculate the intersection counts for all pairs
    intersection_count_at_k = rel_at_k.cumsum(axis=1)

    # we have the same denominator for all ranking vectors
    ranks = np.arange(1, k + 1, 1)

    # Calculating the Precision@K for all Ks for all pairs
    precisions_at_k = intersection_count_at_k / ranks
    # Multiply with the Rel@K for all pairs
    precisions_at_k = precisions_at_k * rel_at_k

    # Calculate the average precisions @ K for all pairs
    average_precisions_at_k = precisions_at_k.mean(axis=1)

    # calculate the final MAP@K
    map_at_k = average_precisions_at_k.mean()

    return map_at_k


class LinkPropMulti:
    def __init__(self, alpha, beta, gamma, delta, rounds, t, k):
        self.rounds, self.k, self.t, self.alpha, self.beta, self.gamma, self.delta = (
            rounds,
            k,
            t,
            alpha,
            beta,
            gamma,
            delta,
        )
        self.user_degrees = None
        self.item_degrees = None
        self.M = None
        self.M_alpha_beta = None
        self.M_gamma_delta = None

    def set_params(self, alpha, beta, gamma, delta, rounds, t, k):
        self.rounds, self.k, self.t, self.alpha, self.beta, self.gamma, self.delta = (
            rounds,
            k,
            t,
            alpha,
            beta,
            gamma,
            delta,
        )

    def fit(self, M):
        # reset model
        self.M_alpha_beta = None
        self.M_gamma_delta = None

        # get node degrees
        if self.user_degrees == None:
            self.user_degrees = M.sum(dim=1)
            self.item_degrees = M.sum(dim=0)

        # exponentiate degrees by model params
        user_alpha = self.user_degrees ** (-self.alpha)
        item_beta = self.item_degrees ** (-self.beta)
        user_gamma = self.user_degrees ** (-self.gamma)
        item_delta = self.item_degrees ** (-self.delta)

        # get rid of inf from 1/0
        user_alpha[torch.isinf(user_alpha)] = 0.0
        item_beta[torch.isinf(item_beta)] = 0.0
        user_gamma[torch.isinf(user_gamma)] = 0.0
        item_delta[torch.isinf(item_delta)] = 0.0

        # to keepe sparsity we can calculate the outer product only for the edges
        # so instead of, outer products: alpha_beta = user_alpha.reshape((-1, 1)) * item_beta
        # and hadamard product: M_alpha_beta = M * alpha_beta
        # we can do:
        # row, col, value = sparse_nonzero(M)
        # self.M_alpha_beta = SparseTensor(
        #     row=row,
        #     col=col,
        #     value=user_alpha[row] * item_beta[col] * value,
        #     sparse_sizes=(M.size(0), M.size(1)),
        # )
        # self.M_gamma_delta = SparseTensor(
        #     row=row,
        #     col=col,
        #     value=user_gamma[row] * item_delta[col] * value,
        #     sparse_sizes=(M.size(0), M.size(1)),
        # )
        self.user_alpha_sparse = SparseTensor(
            row=torch.arange(M.size(0), dtype=torch.long),
            col=torch.zeros(M.size(0), dtype=torch.long),
            value=user_alpha,
            sparse_sizes=(M.size(0), 1),
        ).to(device)
        self.item_beta_sparse = SparseTensor(
            row=torch.zeros(M.size(1), dtype=torch.long),
            col=torch.arange(M.size(1), dtype=torch.long),
            value=item_beta,
            sparse_sizes=(1, M.size(1)),
        ).to(device)
        self.user_gamma_sparse = SparseTensor(
            row=torch.arange(M.size(0), dtype=torch.long),
            col=torch.zeros(M.size(0), dtype=torch.long),
            value=user_gamma,
            sparse_sizes=(M.size(0), 1),
        ).to(device)
        self.item_delta_sparse = SparseTensor(
            row=torch.zeros(M.size(1), dtype=torch.long),
            col=torch.arange(M.size(1), dtype=torch.long),
            value=item_delta,
            sparse_sizes=(1, M.size(1)),
        ).to(device)
        alpha_beta = matmul(self.user_alpha_sparse, self.item_beta_sparse)
        self.M_alpha_beta = sparse_batch_op(2000, torch.multiply, alpha_beta, M)

        gamma_delta = matmul(self.user_gamma_sparse, self.item_delta_sparse)
        self.M_gamma_delta = sparse_batch_op(2000, torch.multiply, gamma_delta, M)

    def fit_multi(self, M, batch_size=600, total=1000):
        self.fit(M)
        for i in range(self.rounds - 1):
            # get top k new links and add them to M
            M_new = M.clone()
            for start in range(0, total, batch_size):
                end = min(start + batch_size, total)
                # calculate how many new links we need to add back for the updated node_degrees
                number_of_links = M.to_dense().to(device).sum()
                k = math.ceil(number_of_links * self.t / (end - start))
                # get top k new links
                predicted_new_link_indices = self.predict_topk(M, start, end, k)
                # add them to M
                indices = np.array([])
                for i, row in enumerate(predicted_new_link_indices):
                    indices = np.concatenate(
                        (indices, list(zip([start + i] * len(row), row))), axis=None
                    )
                indices = indices.reshape(-1, 2)
                row, col, value = M_new.coo()
                M_new = (
                    SparseTensor(
                        row=torch.cat(
                            (
                                row,
                                torch.tensor(indices.T[0], dtype=torch.long),
                            )
                        ),
                        col=torch.cat(
                            (
                                col,
                                torch.tensor(indices.T[1], dtype=torch.long),
                            )
                        ),
                        value=torch.cat((value, torch.ones(len(indices)))),
                        sparse_sizes=(M.size(0), M.size(1)),
                    )
                    .coalesce("max")
                    .to(device)
                )

            # recalculate and store node degrees for next round
            self.user_degrees = M_new.sum(dim=1)
            self.item_degrees = M_new.sum(dim=0)
            self.fit(M)

    def get_preds_for_users(self, X, start, end, block_size=10000):
        # Essentially doing M_alpha_beta.matmul(M.T).matmul(M_gamma_delta),
        # just batched and blocked to control memory allocation
        assert self.M_alpha_beta is not None, "You need to fit the model first"
        return matmul(
            matmul(self.M_alpha_beta[start:end].to(device), X.t().to(device)).to(
                device
            ),
            self.M_gamma_delta.to(device),
        ).to_dense().to(device)

        # left = sparse_blocked_matmul(
        #     self.M_alpha_beta[start:end].to_dense(), X.t(), block_size
        # )
        # right = sparse_blocked_matmul(left, self.M_gamma_delta, block_size)
        # return right


    def predict_topk(self, X, start, end, k):
        """Return top k new links"""
        # user_pred = self.get_preds_for_users(X, start, end).to_dense().to(device)
        user_pred = self.get_preds_for_users(X, start, end).to(device)
        # take observed links out of possible predictions
        user_pred = (
            user_pred - (X[start:end].to_dense().to(device) == 1).float() * 100000
        ).clamp(min=0)
        user_pred = user_pred.topk(k, dim=1)
        # filter out zeros and return indices
        return [
            user_pred.indices[i, user_pred.values[i] > 0].to("cpu").long().numpy()
            for i in range(user_pred.values.size(0))
        ]

    def predict(self, X, batch_size=600):
        user_topk = np.array([], dtype=object)
        for start in tqdm(range(0, X.size(0), batch_size)):
            end = min(start + batch_size, X.size(0))
            user_pred = self.predict_topk(X, start, end, self.k)
            user_topk = np.concatenate((user_topk, user_pred), dtype=object)
        return user_topk

    # def score_ndcg(self, X, y, batch_size=500, total=1000):
    #     # TODO ndcg is on the whole item array of 0s and 1s
    #     user_topk = np.array([])
    #     target_topk = np.array([])
    #     for start in tqdm(range(0, total, batch_size)):
    #         end = min(start + batch_size, total)
    #         user_pred = self.predict_topk(X, start, end, self.k)
    #         target_pred = y[start:end].to_dense().squeeze().topk(self.k, dim=1)
    #         # filter out zeros
    #         target_pred = [
    #             target_pred.indices[i, target_pred.values[i] > 0].to("cpu")
    #             for i in range(target_pred.values.size(0))
    #         ]
    #         user_topk = np.concatenate((user_topk, user_pred))
    #         target_topk = np.concatenate((target_topk, target_pred))

    # return ndcg_score(target_topk, user_topk, k=self.k)

    def score(self, X, y, batch_size=600, total=1000):
        user_topk = np.array([], dtype=object)
        target_topk = np.array([], dtype=object)
        scores = []
        for start in tqdm(range(0, total, batch_size)):
            end = min(start + batch_size, total)
            user_pred = self.predict_topk(X, start, end, self.k)
            target_pred = y[start:end].to_dense().squeeze().topk(self.k, dim=1)
            # filter out zeros
            target_pred = [
                target_pred.indices[i, target_pred.values[i] > 0]
                .to("cpu")
                .long()
                .numpy()
                for i in range(target_pred.values.size(0))
            ]
            user_topk = np.concatenate((user_topk, user_pred), dtype=object)
            target_topk = np.concatenate((target_topk, target_pred), dtype=object)
            scores.append(mean_average_precision(target_pred, user_pred, k=self.k))
            print(np.array(scores).mean())

        return np.array(scores).mean(), user_topk


def predict(M, src, dest, alpha, beta, gamma, delta, rounds, t, batch_size=500):
    linkProp = LinkPropMulti(
        alpha,
        beta,
        gamma,
        delta,
        rounds,
        t,
        k=12,
    )
    linkProp.fit(M)
    preds = linkProp.predict(M, batch_size)
    submission = (
        pd.DataFrame(
            [list(map(lambda x: dest[x], preds[i])) for i in range(len(preds))]
        )
        .fillna(706016001.0).astype('int')
        .astype("string")
    )
    submission["prediction"] = (
        submission[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
        .agg(" 0".join, axis=1)
        .apply(lambda x: "0" + x)
    )
    submission["customer_id"] = src[: len(submission)]
    submission[["customer_id", "prediction"]].to_csv(
        "/content/submission_lp01.csv", index=False
    )


def debug(X, y, alpha, beta, gamma, delta, rounds, t, batch_size=500, total=1000):
    linkProp = LinkPropMulti(
        alpha,
        beta,
        gamma,
        delta,
        rounds,
        t,
        k=12,
    )
    linkProp.fit_multi(X, batch_size, total)
    score, preds = linkProp.score(X, y, batch_size, total)
    print(score)


# find optimal params
def param_search(M):
    param_grid = {
        "alpha": [0.1, 0.3, 0.5, 0.7, 0.9],
        "beta": [0.1, 0.3, 0.5, 0.7, 0.9],
        "gamma": [0.1, 0.3, 0.5, 0.7, 0.9],
        "delta": [0.1, 0.3, 0.5, 0.7, 0.9],
    }
    linkProp = LinkPropMulti(rounds=1, t=0.05, k=12, alpha=0, beta=0, gamma=0, delta=0)
    best = {"score": 0}
    for params in [dict(zip(param_grid, v)) for v in product(*param_grid.values())]:
        val, target = sample_user_items(M, 0.4)
        linkProp.set_params(**params, k=12, rounds=1)
        linkProp.fit(val)
        score = linkProp.score(val, target)
        if score > best["score"]:
            best["score"] = score
            best["params"] = params
        print(score, params)

    # test
    linkProp = LinkPropMulti(rounds=1, t=0.05, k=12, alpha=0, beta=0, gamma=0, delta=0)
    test, target = sample_user_items(M, 0.4)
    params = best["params"]
    linkProp.set_params(**params, k=12, rounds=1)
    linkProp.fit(test)
    score = linkProp.score(val, target)
    print("final score", score, params)


# 0.1, 0.9, 0.3, 0.5 > 0.523
# 0.3, 0.1, 0.1, 0.3 > 0.547
# 0.3, 0.1, 0.3, 0.3 > 0.507
# 0.3, 0.1, 0.3, 0.5 > 0.515
# 0.3, 0.5, 0.7, 0.3 > 0.509
# 0.5, 0.1, 0.9, 0.3 > 0.544
# 0.5, 0.5, 0.9, 0.1 > 0.526
# 0.5, 0.7, 0.7, 0.3 > 0.519


In [ ]:
M, src, dest, src_map, dest_map = load_data()

In [ ]:
M

SparseTensor(row=tensor([      0,       0,       0,  ..., 1371978, 1371978, 1371979]),
             col=tensor([   99, 16003, 16023,  ..., 95506, 99149, 81594]),
             val=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
             size=(1371980, 105542), nnz=27306439, density=0.02%)

In [ ]:
M, src, dest, src_map, dest_map = load_data()
# param_search(M)
# predict(M, src, dest, 0.5, 0.1, 0.9, 0.3)
data, target = sample_user_items(M, 0.1)
# predict(target, src, dest, 0.3, 0.1, 0.1, 0.3)
debug(data, target, 0.3, 0.1, 0.1, 0.3, 1, 0.05, batch_size=5000, total=10000)

AssertionError: ignored

In [ ]:
predict(M[:27400], src, dest, 0.3, 0.1, 0.1, 0.3, 1, 0.05, batch_size=5000)

In [ ]:
!gsutil cp /content/submission_lp2.csv gs://heii-public/

Copying file:///content/submission_lp.csv [Content-Type=text/csv]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/383.4 MiB.                                    


In [ ]:
!KAGGLE_CONFIG_DIR=/content kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f /content/submission_lp2.csv -m "4"

100% 258M/258M [00:17<00:00, 15.6MB/s]
Successfully submitted to H&M Personalized Fashion Recommendations

In [ ]:
submission = pd.read_csv('/content/submission_lp2.csv')

In [ ]:
submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0804028001 0701108001 0812773001 0557048002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0590928022 0601919001 0569248015 0624070002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0573284062 0744136002 0561101001 0804454001 06...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0710510001 0862631001 0856726001 0589556003 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016001 0706016001 0706016001 07...


In [ ]:
submission["prediction"] = submission['prediction'].apply(lambda x: x.replace("tensor(", "").replace(")", ""))

In [ ]:
submission[["customer_id", "prediction"]].to_csv("/content/submission_3.csv", index=False)

In [ ]:
!KAGGLE_CONFIG_DIR=/content kaggle competitions submit -c h-and-m-personalized-fashion-recommendations -f /content/submission_3.csv -m "3"

100% 258M/258M [00:03<00:00, 81.4MB/s]
Successfully submitted to H&M Personalized Fashion Recommendations

In [ ]:
import torch
a = torch.rand((1371980, 105542))
m = torch.rand((1371980, 105542))
b = torch.rand((1371980, 105542))